In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras_tuner
import keras

In [2]:
csv_file_path = './airbnb_dataset/milestone3.csv'
df = pd.read_csv(csv_file_path)
pd.set_option('display.max_columns', None)
df.head()

,log_price,accommodates,bathrooms,host_response_rate,bedrooms,beds,cleaning_fee_True,host_has_profile_pic_t,host_identity_verified_t,instant_bookable_t,property_type_encoded,room_type_encoded,bed_type_encoded,cancellation_policy_encoded,city_encoded,des_sentiment_analysis,name_sentiment_analysis
0,5.010635,-0.072621,-0.404046,0.296013,-0.312048,-0.566461,True,True,True,False,-0.245313,0.876836,0.168067,1.021281,-0.434017,0.216242,-0.435248
1,5.129899,1.783653,-0.404046,0.296013,2.034955,1.027816,True,True,False,True,-0.245338,0.876830,0.168044,1.021245,-0.434042,0.837002,-1.788095
2,4.976734,0.855516,-0.404046,0.296013,-0.312048,1.027816,True,True,True,True,-0.245305,0.876838,0.168074,-0.593873,-0.434009,0.808508,-0.133695
3,6.620073,0.391448,-0.404046,0.296013,0.861454,0.230678,True,True,True,False,0.157557,0.876747,0.167757,-1.013554,2.675098,0.213627,-0.926212
4,4.744932,-0.536689,-0.404046,0.296013,-1.485549,-0.566461,True,True,True,True,-0.245255,0.876851,0.168119,-0.593754,1.412368,-0.101872,-0.095776


In [3]:
features = [col for col in df.columns if col != 'log_price' and df[col].dtype in ['int64', 'float64']]
X = df[features]
y = df['log_price']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Train with NN

In [5]:
# Uncomment below if tensorflow is not installed already
#%pip install tensorflow
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [6]:
# Build Neural Network
nn_model = Sequential([
    Dense(32, activation='relu',input_dim=X_train.shape[1]),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1)
])

nn_model.compile(optimizer='adam', loss='mean_squared_error')

In [7]:
# Train the model
history = nn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20


2024-03-07 01:51:04.697305: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1849/1849 [==============================] - 1s 622us/step - loss: 1.5547 - val_loss: 0.2419
Epoch 2/20
1849/1849 [==============================] - 1s 546us/step - loss: 0.2180 - val_loss: 0.2138
Epoch 3/20
1849/1849 [==============================] - 1s 513us/step - loss: 0.2033 - val_loss: 0.2100
Epoch 4/20
1849/1849 [==============================] - 1s 515us/step - loss: 0.1992 - val_loss: 0.2053
Epoch 5/20
1849/1849 [==============================] - 1s 560us/step - loss: 0.1971 - val_loss: 0.1994
Epoch 6/20
1849/1849 [==============================] - 1s 522us/step - loss: 0.1946 - val_loss: 0.1991
Epoch 7/20
1849/1849 [==============================] - 1s 520us/step - loss: 0.1941 - val_loss: 0.2036
Epoch 8/20
1849/1849 [==============================] - 1s 528us/step - loss: 0.1935 - val_loss: 0.1964
Epoch 9/20
1849/1849 [==============================] - 1s 523us/step - loss: 0.1924 - val_loss: 0.1941
Epoch 10/20
1849/1849 [==============================] - 1s 586us/step - lo

In [8]:
# Evaluate the model
mse = nn_model.evaluate(X_test, y_test)
print(f"Mean Squared Error on Test Set: {mse}")

463/463 [==============================] - 0s 394us/step - loss: 0.1933
Mean Squared Error on Test Set: 0.19332894682884216


### Hyper Tuning

In [47]:
metrics = [keras.metrics.RootMeanSquaredError(name="root_mean_squared_error", dtype=None), keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None)]
def build_hp_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 4)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=128, step=32),
                        activation='relu'))
    model.add(Dense(1))
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mean_squared_error',
                  metrics=metrics)#['mean_squared_error'])
    return model

In [48]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=5,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='checkpoints',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only= False,
    mode='auto'
)

In [49]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_hp_model,
    objective='val_loss',
    max_trials=5,
    seed=10,
    executions_per_trial=3,
    overwrite=True,
    tune_new_entries=True,
    allow_new_entries=True,
    max_consecutive_failed_trials=1
)

In [50]:
# Start searching
tuner_search = tuner.search(X_train, y_train, epochs=15, callbacks=[early_stopping, model_checkpoint], validation_split=0.2)

Trial 5 Complete [00h 00m 48s]
val_loss: 0.18892546991507211

Best val_loss So Far: 0.186910018324852
Total elapsed time: 00h 04m 32s


In [13]:
# View summary of searching
# tuner.results_summary()

In [51]:
# View best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found were: ", best_hyperparameters.values)

Best hyperparameters found were:  {'num_layers': 4, 'units_0': 64, 'units_1': 96, 'learning_rate': 0.0019815016575264635, 'units_2': 32, 'units_3': 96}


In [52]:
best_model = tuner.get_best_models(num_models=1)[0]
mse = best_model.evaluate(X_test, y_test)
print(f"Mean Squared Error on Test Set: {mse}")

463/463 [==============================] - 0s 351us/step - loss: 0.1928 - root_mean_squared_error: 0.4366 - mean_squared_error: 0.1906
Mean Squared Error on Test Set: [0.192816823720932, 0.4365949332714081, 0.19061513245105743]


In [16]:
# View best model
# best_model = tuner.get_best_models()[0]
# best_model.summary()


In [17]:
# Evaluate the best model
# loss, acc = best_model.evaluate(X_test, y_test)
# print('ACC: ', acc)
# print('LOSS: ', loss)